**This script creates image stacks from a folder containing single band .tif images**

In [4]:
import os
import numpy as np
from osgeo import gdal, gdal_array

# Tell GDAL to throw Python exceptions, and register all drivers
gdal.UseExceptions()
gdal.AllRegister()

# Input directory containing single-band images
input_dir = "RCM Outputs"

# Output directory and filename for the new multi-band image
output_dir = "Image Stacks"
output_filename = "RCM_Image_Stack.tif"
output_file = os.path.join(output_dir, output_filename)

# Get list of input image files
image_files = [file for file in os.listdir(input_dir) if file.endswith('.tif')]

# Ensure there are files to process
if not image_files:
    print("No .tif files found in the input directory.")
else:
    # Open the first image to get metadata
    first_image = gdal.Open(os.path.join(input_dir, image_files[0]))
    driver = first_image.GetDriver()
    band_count = len(image_files)
    x_size = first_image.RasterXSize
    y_size = first_image.RasterYSize
    data_type = first_image.GetRasterBand(1).DataType
    projection = first_image.GetProjection()
    geotransform = first_image.GetGeoTransform()

    # Create output multi-band image
    output_ds = driver.Create(output_file, x_size, y_size, band_count, data_type)

    # Set projection and geotransform for the output image
    output_ds.SetProjection(projection)
    output_ds.SetGeoTransform(geotransform)

    # Write data from each input image to corresponding band in the output image
    for i, file in enumerate(image_files):
        image_ds = gdal.Open(os.path.join(input_dir, file))
        band_data = image_ds.GetRasterBand(1).ReadAsArray()
        output_ds.GetRasterBand(i+1).WriteArray(band_data)
        output_ds.GetRasterBand(i+1).SetDescription(os.path.splitext(file)[0])
        image_ds = None  # Close input image

    output_ds = None  # Close output image

    print(f"Multi-band image saved to {output_file} \n")

    # Print the dimensions of the resulting multi-band image
    print(f"Dimensions of the multi-band image: ({band_count}, {y_size}, {x_size})")


Multi-band image saved to Image Stacks/RCM_Image_Stack.tif 

Dimensions of the multi-band image: (20, 3687, 4330)




**This takes all the variable multiband images and creates a single unified image stack from them, clipping it to the extent of a shapefile**



In [17]:
import os
from osgeo import gdal, ogr

# Define user variables
IMAGE_FOLDER = "Image Stacks"
OUTPUT_FOLDER = "Image Stacks/S1 & Terrain"
SHAPEFILE_PATH = "/Users/maciej/Library/CloudStorage/OneDrive-UniversityofGuelph/Grad School/Winter 2024/GEOG 6550/Project Folder/Updated RCM Imagery and Decompositions/Algonquin Park Shapefile/Algonquin_Park_Final.shp"
OUTPUT_FILENAME = "S1_Terrain.tif"

def get_extent_from_shapefile(shapefile):
    # Open the shapefile
    src_ds = ogr.Open(shapefile)
    if src_ds is None:
        raise ValueError(f"Could not open {shapefile}")

    # Get the layer and its extent
    layer = src_ds.GetLayer()
    extent = layer.GetExtent()

    # Close the shapefile
    src_ds = None

    return extent


def create_image_stack(input_files, output_file):
    # Open the first input image to get metadata
    first_image = gdal.Open(input_files[0])
    driver = first_image.GetDriver()
    x_size = first_image.RasterXSize
    y_size = first_image.RasterYSize
    data_type = gdal.GDT_Float32  # Set data type to 32-bit floating point
    projection = first_image.GetProjection()
    geotransform = first_image.GetGeoTransform()

    # Count the total number of non-empty bands among all input images
    total_bands = sum(gdal.Open(file).RasterCount for file in input_files if gdal.Open(file).RasterCount > 0)

    # Create output multi-band image
    output_ds = driver.Create(output_file, x_size, y_size, total_bands, data_type)  # Use 32-bit float data type

    # Set projection and geotransform for the output image
    output_ds.SetProjection(projection)
    output_ds.SetGeoTransform(geotransform)

    # Write data from each input image to corresponding band in the output image
    band_counter = 1  # Start with band 1
    total_written_bands = 0
    for file_index, file in enumerate(input_files):
        image_ds = gdal.Open(file)
        for band_index in range(1, image_ds.RasterCount + 1):
            band_data = image_ds.GetRasterBand(band_index).ReadAsArray()
            # Check if the band is empty (all zeros)
            if not band_data.any():
                print(f"Skipping empty band {band_index} from file {file}")
                continue
            output_ds.GetRasterBand(band_counter).WriteArray(band_data)
            band_name = image_ds.GetRasterBand(band_index).GetDescription() or f"Band_{band_index}"
            output_ds.GetRasterBand(band_counter).SetDescription(f"{os.path.splitext(os.path.basename(file))[0]}_{band_name}")
            band_counter += 1
            total_written_bands += 1
            # Print progress
            print(f"Processed band {total_written_bands}/{total_bands} ({band_name}) of file {file_index + 1}/{len(input_files)}")

        image_ds = None  # Close input image

    output_ds = None  # Close output image



def main():
    # Define input files and output file
    input_files = [
        #os.path.join(IMAGE_FOLDER, "RCM_Image_Stack.tif"),
        #os.path.join(IMAGE_FOLDER, "PALSAR_Yearly_Stack.tif"),
        os.path.join(IMAGE_FOLDER, "S1_Monthly_Image_Stack_2023.tif"),
        os.path.join(IMAGE_FOLDER, "DEM_Image_Stack.tif")
        # Add more input files as needed
    ]
    output_file = os.path.join(OUTPUT_FOLDER, OUTPUT_FILENAME)

    # Check if the output folder exists, if not, create it
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)

    # Get the extent from the shapefile
    extent = get_extent_from_shapefile(SHAPEFILE_PATH)

    # Create image stack
    create_image_stack(input_files, output_file)

    print(f"Multi-band image stack saved to {output_file}")

if __name__ == "__main__":
    main()


Processed band 1/27 (S1_VV_1_2023) of file 1/2
Processed band 2/27 (S1_VH_1_2023) of file 1/2
Processed band 3/27 (S1_VV_2_2023) of file 1/2
Processed band 4/27 (S1_VH_2_2023) of file 1/2
Processed band 5/27 (S1_VV_3_2023) of file 1/2
Processed band 6/27 (S1_VH_3_2023) of file 1/2
Processed band 7/27 (S1_VV_4_2023) of file 1/2
Processed band 8/27 (S1_VH_4_2023) of file 1/2
Processed band 9/27 (S1_VV_5_2023) of file 1/2
Processed band 10/27 (S1_VH_5_2023) of file 1/2
Processed band 11/27 (S1_VV_6_2023) of file 1/2
Processed band 12/27 (S1_VH_6_2023) of file 1/2
Processed band 13/27 (S1_VV_7_2023) of file 1/2
Processed band 14/27 (S1_VH_7_2023) of file 1/2
Processed band 15/27 (S1_VV_8_2023) of file 1/2
Processed band 16/27 (S1_VH_8_2023) of file 1/2
Processed band 17/27 (S1_VV_9_2023) of file 1/2
Processed band 18/27 (S1_VH_9_2023) of file 1/2
Processed band 19/27 (S1_VV_10_2023) of file 1/2
Processed band 20/27 (S1_VH_10_2023) of file 1/2
Processed band 21/27 (S1_VV_11_2023) of file 1/